# Exploring the NIFTY 50 index under different SMA regimes

Markets tend to be manic depressive, going from excessive optimism to excessive pessimism. The distribution of returns tend to be different based on the "regime." A simple way to demarcate these regimes is by using an SMA (Simple Moving Average) to bucket returns into binary groups. Here, we explore how returns are distributed in these buckets.

In [1]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(RPostgres)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)
library(reshape2)
library(ggrepel)
library(lubridate)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")
source("goofy/misc.common.R")

#initialize
indices<-Indices()

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.0     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: xts
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Registered S3 method overwritten by 'xts':
  method     from
  as.zoo.xts zoo 

Attaching package: ‘xts’

The following objects are masked from ‘package:dplyr’:

    first, last

Loading required package: TTR
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 
Version 0.4-0 included new data defaults. See ?getSymbols.

Attaching package: ‘PerformanceAnalytics’

The following object is masked from

In [2]:
smaLb <- 50 #number of days
px <- indices$NseTimeSeries() %>%
    filter(NAME == 'NIFTY 50 TR') %>%
    select(TIME_STAMP, CLOSE) %>%
    collect()

#if we break by years, make sure that they have at least 200 days
numDays <- px %>%
    group_by(Y = year(TIME_STAMP)) %>%
    summarize(N = n())

yrs <- numDays %>%
    filter(N >= 200) %>%
    pull(Y)

In [3]:
pXts <- xts(px$CLOSE, px$TIME_STAMP)
names(pXts) <- c('INDEX')
pXts$SMA <- SMA(pXts[,1], smaLb)
pXts$D_RET <- dailyReturn(pXts$INDEX)
pXts$D_RET_PCT <- pXts$D_RET * 100

pXts <- na.omit(pXts)

In [4]:
statByYear <- data.frame(Y = 0, N_ABOVE = 0, N_BELOW = 0, AVG_ABOVE = 0.0, AVG_BELOW = 0.0, SD_ABOVE = 0.0, SD_BELOW = 0.0)

for(yr in yrs){
    yrStr <- toString(yr)
    numAbove <- as.numeric(sum(ifelse(pXts[yrStr,'INDEX'] > pXts[yrStr,'SMA'], 1, 0)))
    numBelow <- as.numeric(sum(ifelse(pXts[yrStr,'INDEX'] <= pXts[yrStr,'SMA'], 1, 0)))
    
    avgAbove <- as.numeric(mean(pXts[pXts[yrStr,'INDEX'] > pXts[yrStr,'SMA'], 'D_RET_PCT']))
    avgBelow <- as.numeric(mean(pXts[pXts[yrStr,'INDEX'] <= pXts[yrStr,'SMA'], 'D_RET_PCT']))
    
    sdAbove <- as.numeric(sd(pXts[pXts[yrStr,'INDEX'] > pXts[yrStr,'SMA'], 'D_RET_PCT']))
    sdBelow <- as.numeric(sd(pXts[pXts[yrStr,'INDEX'] <= pXts[yrStr,'SMA'], 'D_RET_PCT']))
    
    statByYear <- rbind(statByYear, c(yr, numAbove, numBelow, avgAbove, avgBelow, sdAbove, sdBelow))
}

statByYear <- statByYear[-1,]
print(statByYear)

      Y N_ABOVE N_BELOW   AVG_ABOVE    AVG_BELOW SD_ABOVE SD_BELOW
2  2000     113     137 -0.09308085  0.144475665 1.781884 2.174051
3  2001      94     154 -0.11532862  0.126626476 1.668974 2.197960
4  2002     103     148 -0.08142203  0.121403806 1.583569 2.249262
5  2003     173      81 -0.01570550  0.175604091 2.011063 1.954091
6  2004     170      84 -0.02150497  0.180508624 1.895375 2.177960
7  2005     188      63 -0.06156323  0.335796526 2.071944 1.758617
8  2006     202      48  0.03810886  0.032854986 2.061096 1.774749
9  2007     203      46  0.05129699 -0.037370168 1.948864 2.280878
10 2008      75     171  0.07179554 -0.002680050 2.457022 1.794149
11 2009     187      56  0.04465411 -0.054814571 2.053139 1.947710
12 2010     168      84 -0.01529373  0.153170422 2.092670 1.801981
13 2011      78     169  0.06276224  0.007988055 1.794064 2.109971
14 2012     178      73 -0.03693819  0.221319071 2.077513 1.805035
15 2013     151      99  0.16005759 -0.150441073 1.805919 2.27

The NIFTY got hammered by the GFC in 2008, only to be reflated in 2009. The years prior to do that was a different "regime." Not sure if they can be used in back-tests any more. Let's look at the data from 2010 onwards.

In [5]:
stat200 <- rollapply(pXts["2010/",], 200, function(X){
    numAbove <- as.numeric(sum(ifelse(X[,'INDEX'] > X[,'SMA'], 1, 0)))
    numBelow <- as.numeric(sum(ifelse(X[,'INDEX'] <= X[,'SMA'], 1, 0)))
    
    avgAbove <- as.numeric(mean(X[X[,'INDEX'] > X[,'SMA'], 'D_RET_PCT']))
    avgBelow <- as.numeric(mean(X[X[,'INDEX'] <= X[,'SMA'], 'D_RET_PCT']))
    
    sdAbove <- as.numeric(sd(X[X[,'INDEX'] > X[,'SMA'], 'D_RET_PCT']))
    sdBelow <- as.numeric(sd(X[X[,'INDEX'] <= X[,'SMA'], 'D_RET_PCT']))
    
    xts(matrix(c(numAbove, numBelow, avgAbove, avgBelow, sdAbove, sdBelow), nrow=1), as.Date(last(index(X))))
}, by.column = FALSE)

stat200 <- na.omit(stat200)

print("stat200: HEAD")
print(head(stat200))

print("stat200: TAIL")
print(tail(stat200))

[1] "stat200: HEAD"
           [,1] [,2]      [,3]       [,4]      [,5]     [,6]
2010-10-18  145   55 0.1681451 -0.1253108 0.7920690 1.424586
2010-10-19  145   55 0.1584971 -0.1253108 0.7953032 1.424586
2010-10-20  145   55 0.1473001 -0.1253108 0.7965925 1.424586
2010-10-21  145   55 0.1611645 -0.1253108 0.8126358 1.424586
2010-10-22  145   55 0.1596199 -0.1253108 0.8138340 1.424586
2010-10-25  145   55 0.1665457 -0.1253108 0.8137504 1.424586
[1] "stat200: TAIL"
           [,1] [,2]      [,3]       [,4]      [,5]      [,6]
2019-07-22  131   69 0.1317737 -0.1763620 0.7897486 1.0424609
2019-07-23  131   69 0.1317737 -0.1720788 0.7897486 1.0420135
2019-07-24  131   69 0.1317737 -0.1892885 0.7897486 1.0369850
2019-07-25  131   69 0.1317737 -0.1717176 0.7897486 1.0270232
2019-07-26  131   69 0.1317737 -0.1329976 0.7897486 0.9921091
2019-07-29  131   69 0.1317737 -0.1064821 0.7897486 0.9467772


This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)